In [14]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import gymnasium as gym
from agents import SACAgent2
from omegaconf import DictConfig, OmegaConf
from hydra import compose, initialize
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os
import wandb
import torch as T
from gymnasium.wrappers import RecordVideo
from EnvironmentUtils import LunarEnvHypercubeFabric, LunarEnvRandomFabric, LunarEnvFixedFabric, ValidationHypercube
from agents import SACAgent2, DDPGAgent, SACAgent
import numpy as np
from utils import *
os.environ["SDL_VIDEODRIVER"] = "dummy"
import plotly.express as px
import pandas as pd
import shutil

In [3]:
import wandb
api = wandb.Api()
run = api.run("tum-adlr-ws22-06/ADLR randomized envs/2eqd07v7")

In [4]:
#runs = [...]
# for each run:
#     download best model
#     create directory with validation/run_name
#     create hypercube with 5 points
#     for each hypercube point:
#          create subdirectory
#          make run, record video
#          store reward in result array
# make 3d plotly plots with reward as point color
# make nice gifs, comparing sac2 with and without domain parameters

In [5]:
runs = ["tum-adlr-ws22-06/ADLR randomized envs/2eqd07v7", 
        "tum-adlr-ws22-06/ADLR randomized envs/potkxo0h"]

In [17]:

for run_name in runs:
    res = []
    run = api.run(run_name)
    folder = f"final_validations/{run.name}"
    if os.path.exists(folder):
        shutil.rmtree(folder, ignore_errors=True)
    os.mkdir(folder)
    print(run.name)
    agent = load_best_model(get_agent_from_run_cfg(run.config), run)
    fabric = LunarEnvHypercubeFabric(run.config['training']['pass_env_parameters'], run.config['env'], points_per_axis=5, render_mode='rgb_array') 
    number_points = fabric.number_of_test_points()
    for i in range(number_points):
        env = fabric.generate_env()
        gravity, wind_power, turbulence_power = env.gravity, env.wind_power, env.turbulence_power
        video_path = f"{folder}/gravity_{gravity}_wind_{wind_power}_turbulence_{turbulence_power}/"
        os.mkdir(video_path)
        env = RecordVideo(env, video_folder=video_path)
        obs, _ = env.reset()
        rewards = 0
        while True:

            # Get deterministic action
            with T.no_grad():
                action = agent.action(obs, addNoise=False)
                

            # Take step in environment
            new_obs, reward, done, _, _ = env.step(action)

            # Update obs
            obs = new_obs

            # Update rewards
            rewards += reward

            # End episode if done
            if done:
                break
        print(f"gravity: {gravity}, wind_power: {wind_power}, turbulence_power: {turbulence_power}, reward: {rewards}")
        res.append({"gravity": gravity, "wind_power": wind_power, "turbulence_power": turbulence_power, "reward": rewards})
    df = pd.DataFrame(res)
    df.to_csv(f"{folder}/results.csv", index=False)

sac2_with_env_params


c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning:

WARN: Box bound precision lowered by casting to float32

wandb:   5 of 5 files downloaded.  
c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\wrappers\record_video.py:75: UserWarning:

WARN: Overwriting existing videos at d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-11.999_wind_0_turbulence_0 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)



Best model loaded from .\artifacts\lunar_lander_model-v483 with reward 128.606
Moviepy - Building video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-11.999_wind_0_turbulence_0\rl-video-episode-0.mp4.
Moviepy - Writing video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-11.999_wind_0_turbulence_0\rl-video-episode-0.mp4



c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\wrappers\record_video.py:75: UserWarning:

WARN: Overwriting existing videos at d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-8.999275_wind_0_turbulence_0 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)



Moviepy - Done !
Moviepy - video ready d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-11.999_wind_0_turbulence_0\rl-video-episode-0.mp4
gravity: -11.999, wind_power: 0, turbulence_power: 0, reward: -50.50733376321047
Moviepy - Building video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-8.999275_wind_0_turbulence_0\rl-video-episode-0.mp4.
Moviepy - Writing video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_with_env_params\gravity_-8.999275_wind_0_turbulence_0\rl-video-episode-0.mp4



...

c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\wrappers\record_video.py:75: UserWarning:

WARN: Overwriting existing videos at d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_0.75 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)



Moviepy - Building video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_0.75\rl-video-episode-0.mp4.
Moviepy - Writing video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_0.75\rl-video-episode-0.mp4



c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\wrappers\record_video.py:75: UserWarning:

WARN: Overwriting existing videos at d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_1.0 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)



Moviepy - Done !
Moviepy - video ready d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_0.75\rl-video-episode-0.mp4
gravity: -0.0001, wind_power: 10.0, turbulence_power: 0.75, reward: -209.87352464494865
Moviepy - Building video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_1.0\rl-video-episode-0.mp4.
Moviepy - Writing video d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_1.0\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready d:\Desktop\reps\TUM WS22\tum-adlr-ws22-06\final_validations\sac2_no_env_params\gravity_-0.0001_wind_10.0_turbulence_1.0\rl-video-episode-0.mp4
gravity: -0.0001, wind_power: 10.0, turbulence_power: 1.0, reward: -123.29877900978963


In [8]:
df = pd.DataFrame(res)


In [10]:
df.columns

Index(['gravity', 'wind_power', 'turbulence_power', 'reward'], dtype='object')

In [11]:
fig = px.scatter_3d(df, x='gravity', y='wind_power', z='turbulence_power',
              color='reward')
fig.show()

In [7]:
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    

In [10]:
env_fabric = LunarEnvHypercubeFabric(env_params=cfg.env, pass_env_params=True)
env = env_fabric.generate_env()
env_info = {"input_dims":env.observation_space.shape, "n_actions": env.action_space.shape[0], "max_action": env.action_space.high}

c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [19]:
@widgets.interact_manual(g=(-12.0, 0.0), wind_power=(0.0, 20.0), turbulence_power=(0.0, 2.0)
                         )
def make_video(g=-10, wind_power=0.0, turbulence_power=0.0, plot_every_action=5):
    env = gym.make('LunarLander-v2', continuous=True, render_mode='rgb_array', gravity=g , enable_wind=True, wind_power=wind_power, 
             turbulence_power=turbulence_power)
    obs, info = env.reset()
    step = 0
    while True:
        action = agent.action(obs, addNoise=False)
        new_obs, reward, done, _, _ = env.step(action)
        if step % plot_every_action == 0:
            clear_output(wait=True)
            plt.imshow( env.render())
            plt.show()
        step += 1
        if done:
            break
    env.close()

interactive(children=(FloatSlider(value=-10.0, description='g', max=0.0, min=-12.0), FloatSlider(value=0.0, de…